## Load data

In [1]:
import pandas as pd

url_primclass= 'https://raw.githubusercontent.com/irenekarijadi/RF-LSTM-CEEMDAN/main/Dataset/data%20of%20PrimClass_Jaden.csv'
primclass= pd.read_csv(url_primclass)
data_primclass= primclass[(primclass['timestamp'] > '2015-03-01') & (primclass['timestamp'] < '2015-06-01')]
dfs_primclass=data_primclass['energy']
datas_primclass=pd.DataFrame(dfs_primclass)


## import libraries

In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

from PyEMD import CEEMDAN
import numpy
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics

import time
import dataframe_image as dfi


## Import all functions from another notebook for building prediction method

In [3]:
import Setting
from myfunctions import lr_model,svr_model,ann_model,rf_model,lstm_model,hybrid_ceemdan_rf,hybrid_ceemdan_lstm,proposed_method

## Import parameter settings

In [4]:
hours=Setting.n_hours
data_partition=Setting.data_partition
max_features=Setting.max_features
epoch=Setting.epoch
batch_size=Setting.batch_size
neuron=Setting.neuron
lr=Setting.lr
optimizer=Setting.optimizer

## Run the experiments
### Run this following cell will train and test the proposed method and other benchmark methods on Primary Classroom Dataset

In [5]:
#Linear Regression

start_time = time.time()
lr_primclass=lr_model(datas_primclass,hours,data_partition)
lr_time_primclass=time.time() - start_time
print("--- %s seconds - Linear Regression- primclass ---" % (lr_time_primclass))

#Support Vector Regression
start_time = time.time()
svr_primclass=svr_model(datas_primclass,hours,data_partition)
svr_time_primclass=time.time() - start_time
print("--- %s seconds - Support Vector Regression- primclass ---" % (svr_time_primclass))


#ANN
start_time = time.time()
ann_primclass=ann_model(datas_primclass,hours,data_partition)
ann_time_primclass=time.time() - start_time
print("--- %s seconds - ANN- primclass ---" % (ann_time_primclass))

#random forest
start_time = time.time()
rf_primclass=rf_model(datas_primclass,hours,data_partition,max_features)
rf_time_primclass=time.time() - start_time
print("--- %s seconds - Random Forest- primclass ---" % (rf_time_primclass))

#LSTM
start_time = time.time()
lstm_primclass=lstm_model(datas_primclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
lstm_time_primclass=time.time() - start_time
print("--- %s seconds - lstm- primclass ---" % (lstm_time_primclass))


#CEEMDAN RF
start_time = time.time()
ceemdan_rf_primclass=hybrid_ceemdan_rf(dfs_primclass,hours,data_partition,max_features)
ceemdan_rf_time_primclass=time.time() - start_time
print("--- %s seconds - ceemdan_rf- primclass ---" % (ceemdan_rf_time_primclass))

#CEEMDAN LSTM
start_time = time.time()
ceemdan_lstm_primclass=hybrid_ceemdan_lstm(dfs_primclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
ceemdan_lstm_time_primclass=time.time() - start_time
print("--- %s seconds - ceemdan_lstm- primclass ---" % (ceemdan_lstm_time_primclass))


#proposed method
start_time = time.time()
proposed_method_primclass=proposed_method(dfs_primclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
proposed_method_time_primclass=time.time() - start_time
print("--- %s seconds - proposed_method- primclass ---" % (proposed_method_time_primclass))



--- 0.3186936378479004 seconds - Linear Regression- primclass ---
--- 0.17351269721984863 seconds - Support Vector Regression- primclass ---
--- 0.8292701244354248 seconds - ANN- primclass ---
--- 1.2147679328918457 seconds - Random Forest- primclass ---
--- 7.8936872482299805 seconds - lstm- primclass ---
--- 42.31880259513855 seconds - ceemdan_rf- primclass ---
--- 98.64764738082886 seconds - ceemdan_lstm- primclass ---
--- 95.2485363483429 seconds - proposed_method- primclass ---


## Summarize of experimental results with running time
### Run this following cell will summarize the result and generate output used in Section 4.4 (Table 3) for Primary Classroom dataset

In [6]:
running_time_primclass=pd.DataFrame([lr_time_primclass,svr_time_primclass,ann_time_primclass,
                                   rf_time_primclass,lstm_time_primclass,ceemdan_rf_time_primclass,
                                   ceemdan_lstm_time_primclass,proposed_method_time_primclass])
running_time_primclass=running_time_primclass.T
running_time_primclass.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']


proposed_method_primclass_df=proposed_method_primclass[0:3]
result_primclass=pd.DataFrame([lr_primclass,svr_primclass,ann_primclass,rf_primclass,lstm_primclass,ceemdan_rf_primclass,
                    ceemdan_lstm_primclass,proposed_method_primclass_df])
result_primclass=result_primclass.T
result_primclass.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
primclass_summary=pd.concat([result_primclass,running_time_primclass],axis=0)

primclass_summary.set_axis(['MAPE(%)', 'RMSE','MAE','running time (s)'], axis='index')

primclass_summary.style.set_caption("primclass Results")

index = primclass_summary.index
index.name = "Primary Classroom results"
primclass_summary

,LR,SVR,ANN,RF,LSTM,CEEMDAN RF,CEEMDAN LSTM,Proposed Method
Primary Classroom results,,,,,,,,
MAPE(%),15.508401,14.671384,14.463561,18.396333,16.652353,10.943504,8.550394,7.164047
RMSE,1.040299,1.150466,0.902013,1.342462,0.957101,0.749697,0.523754,0.466712
MAE,0.650500,0.646782,0.577659,0.785441,0.622479,0.455436,0.341895,0.298744
running time (s),0.318694,0.173513,0.829270,1.214768,7.893687,42.318803,98.647647,95.248536


In [7]:
#export table to png
#dfi.export(primclass_summary,"primclass_summary_table.png")

## Calculate percentage improvement
### Run this following cell will calculate percentage improvement and generate output used in Section 4.4 (Table 4) for Primary Classroom dataset

In [10]:
pMAPE_LR_vs_Proposed_primclass=((lr_primclass[0]-proposed_method_primclass[0])/lr_primclass[0])*100
pRMSE_LR_vs_Proposed_primclass=((lr_primclass[1]-proposed_method_primclass[1])/lr_primclass[1])*100
pMAE_LR_vs_Proposed_primclass=((lr_primclass[2]-proposed_method_primclass[2])/lr_primclass[2])*100

pMAPE_SVR_vs_Proposed_primclass=((svr_primclass[0]-proposed_method_primclass[0])/svr_primclass[0])*100
pRMSE_SVR_vs_Proposed_primclass=((svr_primclass[1]-proposed_method_primclass[1])/svr_primclass[1])*100
pMAE_SVR_vs_Proposed_primclass=((svr_primclass[2]-proposed_method_primclass[2])/svr_primclass[2])*100

pMAPE_ANN_vs_Proposed_primclass=((ann_primclass[0]-proposed_method_primclass[0])/ann_primclass[0])*100
pRMSE_ANN_vs_Proposed_primclass=((ann_primclass[1]-proposed_method_primclass[1])/ann_primclass[1])*100
pMAE_ANN_vs_Proposed_primclass=((ann_primclass[2]-proposed_method_primclass[2])/ann_primclass[2])*100

pMAPE_RF_vs_Proposed_primclass=((rf_primclass[0]-proposed_method_primclass[0])/rf_primclass[0])*100
pRMSE_RF_vs_Proposed_primclass=((rf_primclass[1]-proposed_method_primclass[1])/rf_primclass[1])*100
pMAE_RF_vs_Proposed_primclass=((rf_primclass[2]-proposed_method_primclass[2])/rf_primclass[2])*100

pMAPE_LSTM_vs_Proposed_primclass=((lstm_primclass[0]-proposed_method_primclass[0])/lstm_primclass[0])*100
pRMSE_LSTM_vs_Proposed_primclass=((lstm_primclass[1]-proposed_method_primclass[1])/lstm_primclass[1])*100
pMAE_LSTM_vs_Proposed_primclass=((lstm_primclass[2]-proposed_method_primclass[2])/lstm_primclass[2])*100

pMAPE_ceemdan_rf_vs_Proposed_primclass=((ceemdan_rf_primclass[0]-proposed_method_primclass[0])/ceemdan_rf_primclass[0])*100
pRMSE_ceemdan_rf_vs_Proposed_primclass=((ceemdan_rf_primclass[1]-proposed_method_primclass[1])/ceemdan_rf_primclass[1])*100
pMAE_ceemdan_rf_vs_Proposed_primclass=((ceemdan_rf_primclass[2]-proposed_method_primclass[2])/ceemdan_rf_primclass[2])*100


pMAPE_ceemdan_lstm_vs_Proposed_primclass=((ceemdan_lstm_primclass[0]-proposed_method_primclass[0])/ceemdan_lstm_primclass[0])*100
pRMSE_ceemdan_lstm_vs_Proposed_primclass=((ceemdan_lstm_primclass[1]-proposed_method_primclass[1])/ceemdan_lstm_primclass[1])*100
pMAE_ceemdan_lstm_vs_Proposed_primclass=((ceemdan_lstm_primclass[2]-proposed_method_primclass[2])/ceemdan_lstm_primclass[2])*100


df_PI_primclass=[[pMAPE_LR_vs_Proposed_primclass,pMAPE_SVR_vs_Proposed_primclass,pMAPE_ANN_vs_Proposed_primclass,
                pMAPE_RF_vs_Proposed_primclass,pMAPE_LSTM_vs_Proposed_primclass,pMAPE_ceemdan_rf_vs_Proposed_primclass,
                pMAPE_ceemdan_lstm_vs_Proposed_primclass],
                [pRMSE_LR_vs_Proposed_primclass,pRMSE_SVR_vs_Proposed_primclass,pRMSE_ANN_vs_Proposed_primclass,
                pRMSE_RF_vs_Proposed_primclass,pRMSE_LSTM_vs_Proposed_primclass,pRMSE_ceemdan_rf_vs_Proposed_primclass,
                pRMSE_ceemdan_lstm_vs_Proposed_primclass],
                [pMAE_LR_vs_Proposed_primclass,pMAE_SVR_vs_Proposed_primclass,pMAE_ANN_vs_Proposed_primclass,
                pMAE_RF_vs_Proposed_primclass,pMAE_LSTM_vs_Proposed_primclass,pMAE_ceemdan_rf_vs_Proposed_primclass,
                pMAE_ceemdan_lstm_vs_Proposed_primclass]]

PI_primclass=pd.DataFrame(df_PI_primclass, columns=["Proposed Method vs.LR", "Proposed Method vs.SVR"," Proposed Method vs.ANN",
                                      "Proposed Method vs.RF","Proposed Method vs.LSTM","Proposed Method vs.CEEMDAN RF",
                                      "Proposed Method vs. CEEMDAN LSTM"])
PI_primclass= PI_primclass.round(decimals = 2)
PI_primclass.set_axis(['MAPE(%)', 'RMSE','MAE'], axis='index')
PI_primclass.style.set_caption("Percentage Improvement-primary classroom Building")

index = PI_primclass.index
index.name = "Percentage Improvement primclass"
PI_primclass

,Proposed Method vs.LR,Proposed Method vs.SVR,Proposed Method vs.ANN,Proposed Method vs.RF,Proposed Method vs.LSTM,Proposed Method vs.CEEMDAN RF,Proposed Method vs. CEEMDAN LSTM
Percentage Improvement primclass,,,,,,,
MAPE(%),53.81,51.17,50.47,61.06,56.98,34.54,16.21
RMSE,55.14,59.43,48.26,65.23,51.24,37.75,10.89
MAE,54.07,53.81,48.28,61.96,52.01,34.40,12.62


In [ ]:
#export table to png
#dfi.export(PI_primclass,"PI_primclass_table.png")